# 똑똑한 챗봇 만들기

이 세션에서는 코사인 유사성 알고리즘을 사용하여 챗봇을 만들 것입니다.

챗봇의 기본 개념은 간단합니다. 특정 입력을 인식하고 의미 있는 응답으로 회신할 챗봇을 만들 수 있기를 원합니다. 

먼저 챗봇 의 데이터베이스를 구축하고, 선택한 알고리즘을 적용한 다음, Jupyter 노트북에서 채팅 을 위한 간단한 대화형 대시보드를 만들 것입니다!

이 연습은 Parul Pandey의 [위대한 작품](https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e)에서 영감을 받았습니다.

## 1.  챗봇에 지식 기반 추가

챗봇의 대화 및 상호 작용 능력은 사용자가 입력한 데이터에 따라 달라집니다. 따라서 첫 번째 단계는 챗봇을 계획하는 것입니다. 특정 작업에 대한 올바른 지식 기반을 선택해야합니다. 예를 들어, 여러분이 스포츠카에 관한 질문에 답하기를 원한다면 영화 리뷰로 챗봇을 훈련시키지 않을 것입니다!

이 연습에서는 챗봇에 대한 지식을 원하는 항목을 결정하고 필요한 데이터 집합을 찾습니다.

우선 단순히 챗봇의 위키백과 페이지(https://en.wikipedia.org/wiki/Chatbot)를 복사하여 텍스트 파일에 붙여넣은 robots.txt 문서를 포함 시켰습니다.
  
당신의 챗봇이 어떤 질문에 답하길 원하십니까? 멸종 위기에 처한 종, 굶주림, 양성 평등, 청정 에너지 또는 원하는 다른 주제에 대한 챗봇을 훈련시킬 수 있습니다! 이전의 경험에서 배운 도구를 사용해서 정보를 수집하고 지식 라이브러리를 구축하세요. 

### 실습: 어떤 챗봇을 만들고 싶으세요? 브레인스토밍 시간을 갖고 여러분의 아이디어를 여러분의 그룹과 공유하세요!

자, 이제 시작합시다!

### 1.1 라이브러리 가져오기

이 연습에서는 다음과 같은 라이브러리가 필요합니다.:

In [1]:
import nltk # 텍스트 데이터를 처리하기 위한 것
import numpy as np # 말뭉치를 배열로 나타내기 위한 것
import random 
import string # 표준 파이썬 문자열을 처리
from sklearn.metrics.pairwise import cosine_similarity # 이를 나중에 사용하여 두 개의 문장이 얼마나 비슷한지를 결정합니다.
from sklearn.feature_extraction.text import TfidfVectorizer # Experience 2에서 단어 가방을 만드는 함수를 만들었던 것을 기억하십니까? 이 함수는 같은 일을 합니다!

### 1.2 데이터 처리

이제 지식 기반을 처리하기 시작했습니다. 텍스트 파일을 읽고, 모든 것을 소문자로 변환하고 토큰화할 것입니다. 여러분은 토큰화가 무엇을 하는지 기억하십니까?  
  
문서를 토큰으로 분할하는 데 도움이 되는 사전 교육된 토큰화 프로그램이 있습니다.

이러한 사전 교육된 토큰화 프로그램을 사용하는 것이 처음이라면 `nltk.download('punkt')` 명령으로 다운로드해야합니다. 또한 미리 정의된 모델을 사용하여 표제어 추출(lemmatization)을 수행할 것입니다. 이것은 `nltk.download('wordnet')`를 사용하여 다운로드됩니다. 이후에 셀을 다시 실행해야 하는 경우 줄에 주석을 달 수 있습니다. 


#### 선택 사항: 필요한 기능이 없는 경우 아래 코드 블록을 실행합니다.

In [2]:
nltk.download('punkt')   # 토큰화 프로그램 처음 사용
nltk.download('wordnet') # 표제어 추출 프로그램 처음 사용

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### 1.2.1 파일 열기

먼저 open() 함수를 사용하여 .txt 파일을 로드합니다.,이어서 .read()를 사용하여 데이터를 raw_data라고 부르는 변수에 로드합니다.

In [3]:
'''
데이터를 수집한 경우, 아래의 변수 'filepath' 를 지식 기반의 위치로 업데이트한다. 
지식 기반은 텍스트 파일의 문장으로 구성되어야 합니다.
'''
filepath='./data/robots.txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data=corpus.read()
print (raw_data)


Chatbot
From Wikipedia, the free encyclopedia
Jump to navigationJump to search
A chatbot (also known as a smartbot, talkbot, chatterbot, Bot, IM bot, interactive agent, Conversational interface or Artificial Conversational Entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods.[1] Such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the Turing test. Chatbots are typically used in dialog systems for various practical purposes including customer service or information acquisition. Some chatbots use sophisticated natural language processing systems, but many simpler ones scan for keywords within the input, then pull a reply with the most matching keywords, or the most similar wording pattern, from a database.

The term "ChatterBot" was originally coined by Michael Mauldin (creator of the first Verbot, Julia) in 1994 to describe these conversational

#### 1.2.2 소문자로 변환

모든 텍스트를 먼저 소문자로 변환합니다. 결과가 완료되면 검사해야 합니다.

In [4]:
#your code here
raw_data = raw_data.lower().replace("\n", "")
print(raw_data)

chatbotfrom wikipedia, the free encyclopediajump to navigationjump to searcha chatbot (also known as a smartbot, talkbot, chatterbot, bot, im bot, interactive agent, conversational interface or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods.[1] such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the turing test. chatbots are typically used in dialog systems for various practical purposes including customer service or information acquisition. some chatbots use sophisticated natural language processing systems, but many simpler ones scan for keywords within the input, then pull a reply with the most matching keywords, or the most similar wording pattern, from a database.the term "chatterbot" was originally coined by michael mauldin (creator of the first verbot, julia) in 1994 to describe these conversational progr

#### 1.2.3  문장 세분화

funtion.sent_tokenizer를 사용하여 문서를 문장 목록으로 변환합니다.

In [5]:
#your code here
from nltk.tokenize import sent_tokenize

sent_tokens = sent_tokenize(raw_data)
print(sent_tokens)

['chatbotfrom wikipedia, the free encyclopediajump to navigationjump to searcha chatbot (also known as a smartbot, talkbot, chatterbot, bot, im bot, interactive agent, conversational interface or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods.', '[1] such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the turing test.', 'chatbots are typically used in dialog systems for various practical purposes including customer service or information acquisition.', 'some chatbots use sophisticated natural language processing systems, but many simpler ones scan for keywords within the input, then pull a reply with the most matching keywords, or the most similar wording pattern, from a database.the term "chatterbot" was originally coined by michael mauldin (creator of the first verbot, julia) in 1994 to describe these conversa

좋아요! 문서를 문장으로 구분했습니다. 그러나 지금 개별 문장을 해독하는 것은 어렵다는 것을 알 수 있습니다.

### 실습: 각 문장을 새로운 줄에 출력합니다!

이렇게 하면 문장을 읽는 것이 더 쉬울 것입니다!

이를 수행하는 방법을 학습하려면 [여기](https://www.geeksforgeeks.org/print-lists-in-python-4-different-ways/) 링크를 사용하십시오.


In [6]:
#your code here
print("printing lists in new line") 
  
print(*sent_tokens, sep = "\n")

printing lists in new line
chatbotfrom wikipedia, the free encyclopediajump to navigationjump to searcha chatbot (also known as a smartbot, talkbot, chatterbot, bot, im bot, interactive agent, conversational interface or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods.
[1] such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the turing test.
chatbots are typically used in dialog systems for various practical purposes including customer service or information acquisition.
some chatbots use sophisticated natural language processing systems, but many simpler ones scan for keywords within the input, then pull a reply with the most matching keywords, or the most similar wording pattern, from a database.the term "chatterbot" was originally coined by michael mauldin (creator of the first verbot, julia) in 1994 to describ

#### 1.2.4 단어 토큰화

문장을 단어 목록으로 변환하기 위해 funtion.word_tokenizer를 사용합니다.

In [7]:
#your code here
from nltk.tokenize import word_tokenize

word_tokens = word_tokenize(raw_data)
print(word_tokens)

['chatbotfrom', 'wikipedia', ',', 'the', 'free', 'encyclopediajump', 'to', 'navigationjump', 'to', 'searcha', 'chatbot', '(', 'also', 'known', 'as', 'a', 'smartbot', ',', 'talkbot', ',', 'chatterbot', ',', 'bot', ',', 'im', 'bot', ',', 'interactive', 'agent', ',', 'conversational', 'interface', 'or', 'artificial', 'conversational', 'entity', ')', 'is', 'a', 'computer', 'program', 'or', 'an', 'artificial', 'intelligence', 'which', 'conducts', 'a', 'conversation', 'via', 'auditory', 'or', 'textual', 'methods', '.', '[', '1', ']', 'such', 'programs', 'are', 'often', 'designed', 'to', 'convincingly', 'simulate', 'how', 'a', 'human', 'would', 'behave', 'as', 'a', 'conversational', 'partner', ',', 'thereby', 'passing', 'the', 'turing', 'test', '.', 'chatbots', 'are', 'typically', 'used', 'in', 'dialog', 'systems', 'for', 'various', 'practical', 'purposes', 'including', 'customer', 'service', 'or', 'information', 'acquisition', '.', 'some', 'chatbots', 'use', 'sophisticated', 'natural', 'lang

이제 우리는 토큰을 가지고 있습니다!

토큰을 검사하여 예상한 대로 보이는지 확인하고, 우리의 챗봇을 교육하기에 충분한 정보가 있는지 확인하는 것이 좋습니다. 각 문장 토큰은 지식 기반에 대한 단일 정보를 제공하므로 하나의 지식으로 간주될 수 있습니다. 지식 기반의 크기는 챗봇이 얼마나 능력이 있는지를 결정할 수 있습니다.

문장의 길이와 단어 토큰을 인쇄하는 코드는 아래와 같습니다.

In [8]:
# 이 기능을 사용하여 지식 기반의 크기를 확인하십시오. 최소 150개의 문장 토큰이 있어야 하며, 가능하면 최대 1000개까지 가능합니다.
# 챗봇에 충분한 컨텍스트를 제공할 수 있습니다.

#your code here
print(len(sent_tokens), len(word_tokens))

172 3601


잘했습니다! 이제 챗봇에 전달되는 정보의 양을 파악할 수 있습니다. 만약 여러분이 충분한 문장 토큰을 가지고 있지 않다는 것을 알게 된다면, 지금이 다시 나가서 더 많은 정보를 찾을 때입니다!
  
#### 1.2.5 표제어 추출

우리는 다운로드 한 WordNetleMmatizer를 사용하여 단어 토큰에서 표제어를 추출합니다.

In [9]:
lemmer = nltk.stem.WordNetLemmatizer() # lemmer 클래스를 초기화합니다. WordNet은 NLTK에 포함된 의미 론적 영어 사전입니다
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

#### 1.2.6 정규화

또한 지식 기반에 유용하지 않으므로 구두점을 제거하는 함수를 작성합니다.

In [10]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict))) #see previous section 1.2.5 lemmatization

이 함수들이 어떻게 작동하는지 볼 시간입니다! 다음 문장에서 함수를 호출하고 결과를 출력합니다.  함수가 테스트 문장에 어떻게 동작합니까?

In [11]:
test_sentence='Today was a wonderful day. The sun was shining so brightly and the birds were chirping loudly!'
# your code here
test_word_tokens = nltk.word_tokenize(test_sentence)  # 문서를 단어 목록으로 변환

lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

LemTokens(test_word_tokens)

['Today',
 'wa',
 'a',
 'wonderful',
 'day',
 '.',
 'The',
 'sun',
 'wa',
 'shining',
 'so',
 'brightly',
 'and',
 'the',
 'bird',
 'were',
 'chirping',
 'loudly',
 '!']

## 2. 일치하는 항목과 코사인 유사성

축하합니다. 지식 데이터베이스를 텍스트에서 문장과 토큰으로 변환했습니다! 다음 단계가 무엇인지 기억합니까? 컴퓨터는 숫자를 잘 처리 할 수 있으므로, 토큰을 숫자로 변환할 것입니다!

이를 위해, 데이터 획득과 실험 2 단계에서 배운 단어 가방과 tf-idf를 다시 살펴볼 것입니다.
문서 벡터를 사용하면 챗봇을 만드는 데 어떻게 도움이 될까요?

### 2.1 컴퓨터가 어떻게 유사성을 찾을 수 있습니까?

입력을 읽고 입력이 말하는 내용을 '고려'한 다음 가장 의미 있는 것으로 응답할 수 있는 챗봇을 만들고 싶다고 가정해 보십시오. 가장 일반적인 방법 중 하나는 질문을 받고 우리의 질문과 매우 유사한 데이터 세트 내에서 정보를 찾는 것입니다. 예를 들어, 질문에 '컴퓨터 장애'라는 문구가 포함되어 있으면 '컴퓨터 고장' 또는 '하드웨어 장애'와 같은 '컴퓨터 장애'와 유사한 단어가 포함된 문장에 답이 있다고 가정합니다.

### 2.2 [코사인 유사성](https://en.wikipedia.org/wiki/Cosine_similarity) 이란 무엇인가?

데이터 세트를 문서 벡터로 변환 할 것입니다. 코사인 유사성은 유사한 벡터를 찾을 수 있게 해주며, 이러한 벡터는 의미가 비슷하다고 가정할 것입니다. 

### 아래 예제를 살펴보겠습니다!

#### 2.2.1 데이터 세트

여기에 세 문장이 있습니다. 처음 두개의 문장은 우리의 작은 지식 기반이며, 세 번째는 샘플 질문입니다. 세 문장을 문서 벡터로 처리하고 코사인 유사성을 사용하여 지식 기반 중 어느 것이 우질문에 대한 더 나은 응답을 제공할 수 있는지를 확인합니다!

In [12]:
# 두 문장을 지식 기반으로,한 문장을 질문으로 합니다.
Sentence_1='John is my father'
Sentence_2='Jane is my mother'
Question='who is my father'

#### 2.2.2  어휘

어휘는 우리의 문장과 질문에 존재하는 모든 단어의 목록입니다.   

### 실습: 데이터 세트에 포함된 어휘를 나열하십시오.

In [13]:
Vocabulary = ['John', 'is', 'my', 'father', 'Jane', 'mother', 'who']

#### 2.2.3 단어 가방

이것을 어떻게 단어 가방에 넣을까요?

### 실습: 단어 가방 기술을 사용하여 위의 두 문장의 정보를 나타내는 배열을 구성하십시오. 

힌트 1: 배열은 크기 (3,7) 이어야 하며 배열 내의 각 숫자는 행에서 해당 단어의 존재를 표시하기 위해 0또는 1입니다.  
힌트 2: 배열은 함수 array_name = np.array([[row 1],[row 2],[row 3]) 를 사용하여 구성합니다.

In [27]:
# your code here
bag_of_words = np.array([
    [1, 1, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 1, 1, 0],
    [0, 1, 1, 1, 0, 0, 1] 
])

bag_of_words2 = np.array([
    [1, 1, 1, 1, 0, 0, 0],
    [0, 1, 1, 0, 1, 1, 0],
    [0, 1, 1, 0, 0, 1, 1]
])

단어 가방에서 처음 두 행은 문장 1과 문장 2에 포함 된 모든 지식을 인코딩합니다. 세 번째 줄은 질문을 인코딩합니다.
질문에 가장 근접한 지식을 찾으려면 질문과 코사인 유사도가 가장 높은 문장을 찾으면 됩니다.  

#### 2.2.4 코사인 유사성 찾기

처음 시작 부분에서 라이브러리를 가져온 것을 기억하나요? 그 중 하나가 `from sklearn.metrics.pairwise import cosine_similarity` 입니다. cosine_similarity 함수를 사용하면 문장을 비교할 수 있습니다. 각 문장을 어떤 방향으로 가리키는 선인 벡터로 상상해 보십시오. 코사인 유사도는 각 선 사이의 각도를 계산하고 두 선이 유사할수록 각도가 작아지고 코사인 유사도가 높아집니다.

지식 기반의 각 정보와 질문 사이의 코신 유사성을 계산해 보겠습니다. 배열의 첫 번째 행은 문장 1에 해당하며 두 번째 행은 문장 2에 해당합니다. 어떤 문장이 질문과 더 높은 코사인 유사성이 있는지 봅시다!

### 실습: 인덱싱 기능을 사용하여 단어 가방의 상위 2행만 선택합니다.

In [19]:
bag_of_words[:2, :]

array([[1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 0, 1, 1, 0]])

좋습니다. 데이터베이스가 선택되었습니다. 이제 bag_of_words 변수의 마지막 행에 있는 질문을 출력하겠습니다:

In [20]:
print (bag_of_words[-1,:].reshape(1, -1))

[[0 1 1 1 0 0 1]]


위의 두 출력의 차이점은 무엇입니까? 아래 코사인 유사성 연습을 위해 해당하는 행을 사용해 보십시오.

이제 코사인 유사성을 적용할 준비가 되었습니다.

In [30]:
# 우리의 데이테 베이스를 기억하세요.
# 문장_1='John is my father'
# 문장_2='Jane is my mother'
# 질문='who is my father'

cosine_similarity(bag_of_words[:2,:], bag_of_words[-1,:].reshape(1, -1))

array([[0.75],
       [0.5 ]])

#### 이제 결과를 분석하십시오!

어떤 문장이 더 높은 코사인 유사성을 가지고 있나요? 예상한 답이었나요?

만약 당신의 챗봇이 코사인 유사도 점수가 가장 높은 문장을 정답으로 돌려주도록 했다면, 질문에 정확하게 답했을까요? 그것이 기본적으로 챗봇이 질문에 대답하는 방법입니다!
  
### 실습: 이제는 문장 2가 더 높은 코사인 유사성을 갖는 질문을 생각할 수 있습니까? 우리에게 점수를 보여주세요!

In [29]:
# 초기 문장과 단어 가방의 결과에 대해 업데이트 합니다.
# 단어 가방을 업데이트 합니다. 이 질문은 다음과 같습니다: ‘Who is my mother?’
Question = 'Who is my mother?'

# Your code here
cosine_similarity(bag_of_words2[:2,:], bag_of_words[-1,:].reshape(1, -1))

array([[0.75],
       [0.5 ]])

축하합니다! 이제 챗봇에 뇌가 생겼으니 입을 만들어 봅시다!

## 3. 챗봇이 첫 마디를 말하도록 하세요.

### 3.1 인사말
모든 대화가 시작될 때 챗봇은 인사말을 기대할 수 있습니다. 이러한 인사말은 질문이 아니지만 챗봇도 인사에 대한 답변이 있어야합니다. 우리는 챗봇이 받을 것으로 예상되는 몇 가지의 일반적인 인사말을 입력하고, 챗봇이 무작위로 선택한 인사말로 응답하도록 할 수 있습니다.
   
팁: 챗봇에 몇 가지 경고가 표시될 수 있습니다. 챗봇이 합리적인 답변을 제공하는 한 그것들에 대해 걱정하지 마세요.

#### 3.1.1 입력 및 응답 목록 작성

먼저 챗봇이 가질 인사말 목록을 만들어 보겠습니다.

In [31]:
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey", "hey there"]
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

### 실습: 인사말 입력 및 응답을 추가하여 챗봇을 개인화합니다.

In [ ]:
# Your code here

#### 3.1.2 인사말을 수신하고 반환하는 함수 만들기

In [32]:
def greeting(sentence):
    for word in sentence.split(): # 문장의 각 단어를 살펴봅니다.
        if word.lower() in GREETING_INPUTS: # 단어가 GREETING_INPUT와 일치하는지 확인합니다.
            return random.choice(GREETING_RESPONSES) # Greeting_Response로 답장합니다.

테스트해 봅시다! 다양한 유형의 인사말을 챗봇에 입력하고 어떻게 응답하는지 확인하십시오.  인사말을 여러 번 실행합니다. 매번 같은 답변이 나오나요?
힌트: ctrl-enter를 누르면 선택된 셀을 실행할 수 있습니다. ctrl 키를 누른 상태에서 Enter 키를 여러 번 누르면 동일한 셀을 반복적으로 실행할 수 있습니다.

In [49]:
# Your code here
greeting("Hi nice to meet you !")

'hi'

#### 3.1.3 질문을 받고 답변을 반환하는 함수 만들기

이제 누군가가 로봇에게 질문을 할 때 반응을 계산하는 함수를 정의하겠습니다.

응답 함수:
1. 질문을 받아들입니다.
2. 코사인 유사성을 사용하여 질문에 가장 가까운 문장을 찾습니다.
3. 답변으로 해당 문장을 반환합니다.

챗봇이 전혀 쓸모없는 답안을 되돌려주는 것을 막기 위해 코사인 유사도가 0보다 큰 경우에만 답변을 반환합니다. 이 경우 챗봇은 단순히 질문을 이해하지 못한다고 말할 것입니다.

In [76]:
def response(user_response):
    """
    유저의 응답을 인자값으로 받아 문장 토큰 리스트에 추가하고, 단어 빈도를 계산하여 tfidf 값을 구한 후, \n
    코사인 유사도 값을 책정합니다. 
    """
    
    robo_response='' # 문자열을 포함하도록 변수를 초기화
    sent_tokens.append(user_response) # sent_messages에 사용자 응답 추가
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english') # 단어빈도 계산
    tfidf = TfidfVec.fit_transform(sent_tokens) # tfidf 값 가져오기
    vals = cosine_similarity(tfidf[-1], tfidf) # 코사인 유사성 값 가져오기(사용자의 문장과 tfidf 벡터 간의 코사인 유사도 계산)
    idx = vals.argsort()[0][-2] # 유사도 값을 정렬해서 질문 다음으로 가장 높은 유사도값을 가져와서
    flat = vals.flatten() 
    flat.sort() # 오름차순으로 정렬
    req_tfidf = flat[-2] 
    
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

#### 3.1.4 응답 함수 테스트

응답 함수를 테스트합니다. response() 함수에 몇 가지 질문을 작성하고 결과를 확인합니다. 원하는 답변에 근접해 있나요?

계속해서 'I'm sorry! I don't understand you'라는 답변을 받는다면, 지식 기반에 더 많은 정보를 추가해야 할 수도 있습니다.

In [84]:
import warnings
warnings.filterwarnings("ignore")

# Your code here
response("What is the chatter bot?")

'What is the chatter bot?'

#### 3.1.5  지식 기반에 더 많은 정보 추가

지식 기반에 수동으로 더 많은 정보를 추가하거나 검색하고 텍스트 파일에 더 많은 데이터를 추가해서 특정한 답을 얻을 수 있습니다!

## 4.똑똑한 챗봇을 테스트해보세요!

#### 4.1.1 챗봇 테스트
이제 챗봇을 테스트할 시간입니다. 이 챗봇은 주피터 노트북 안에서만 실행됩니다. 명령에 대한 특정 작업을 수행하도록 챗봇을 훈련시킬 수 있습니다.

예를 들면 다음과 같습니다:
1. 'bye' 라고 말하면 챗봇이 종료됩니다.
2. 인사말 문구 중 하나인 응답을 제공하면 챗봇이 응답으로 인사말을 제공합니다.  

특정 동작을 트리거하는 키워드 목록에 따라 챗봇에 기능을 추가할 수 있습니다. 예를 들어 챗봇을 통해 시간을 알려주거나, 농담을 하거나, 사진을 인쇄할 수도 있습니다.

기본 챗봇은 아래와 같습니다. 지식 기반에서 질문을 얼마나 잘 이해할 수 있는지 확인하기 위해 이 작업을 수행할 수 있습니다. 챗봇을 훈련하기 위해서는 며칠이 걸리는 수천 개의 문장이 필요하기 때문에 아직은 질문에 잘 대답할 수 없을 것입니다. 그러나 간단한 질문에는 대답할 수 있어야합니다!

#### 4.1.2 챗봇에 시간 기능 추가

이제 챗봇에 더 많은 기능을 추가하여 기능을 향상시킬 수 있습니다! 먼저, 사용자가 '시간'을 입력하면 시간을 알려주는 기능을 챗봇에 추가합니다.

힌트: 위의 인사말 함수와 유사한 템플릿을 사용합니다. datetime 라이브러리를 가져와서 datetime.datetime.now()을 호출하여 현재 시간 정보를 얻을 수 있습니다.

In [85]:
import datetime

def tell_time(sentence):
    for word in sentence.split():
        if word.lower() == 'time':
            currentdt = datetime.datetime.now()
            return currentdt.strftime("%Y-%m-%d %H:%M:%S")

tell_time('time')

'2023-11-17 11:33:55'

### 지금 챗봇을 테스트해보세요!

In [89]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
                # tell_time 함수를 작성했으면 아래의 문장을 주석해제하세요.
            if(tell_time(user_response)!=None):
                print("ROBO: "+tell_time(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
ROBO: [39]designthe chatbot design is the process that defines the interaction between the user and the chatbot.
ROBO: virtual agent chat.
ROBO: [39]designthe chatbot design is the process that defines the interaction between the user and the chatbot.
ROBO: [39]designthe chatbot design is the process that defines the interaction between the user and the chatbot.
ROBO: [39]designthe chatbot design is the process that defines the interaction between the user and the chatbot.
ROBO: You are welcome..


이제 선택한 데이터에 대해 교육을 받을 수 있는 매우 기본적인 챗봇이 있습니다. 챗봇을 다음과 같은 방법으로 개선하십시오:
1. 인사말 입력 및 인사말 응답 수를 늘립니다.
2. 작별 인사를 할 단어(예: see you!, unit, exit)를 늘립니다.
3. 다른 주제에 대해 챗봇을 교육하거나, 데이터를 추가하여 챗봇의 지식 기반에 추가합니다.